## nb3: species tree inference

In this notebook we will simulate sequence data to test the ability of several phylogenetic inference methods to infer the true species tree topology when high levels of genealogical discordance are present. We will use `toytree` to setup the simulation scenario, `ipcoal` to simulate genealogical variation and sequence data, and the `ipyrad-analysis (ipa)` toolkit to serve as a wrapper to call several phylogenetic inference software tools.

In [28]:
import toytree
import ipcoal
import ipyrad.analysis as ipa

### Phylogenomic inference methods (tetrad quartet species tree inference)

Large multi-locus datasets are typically analyzed in one of three ways to infer a phylogenetic estimate efficiently. This notebook wil compare the three approaches below.

<img src="https://eaton-lab.org/slides/data-svg/consensus-pre.svg" style="width:85%">

### The true species tree

The imbalanced (comb-shaped) tree topology below represents the true species tree history that we hope to infer from sequence data. By setting demographic parameters on this species tree history we can create a difficult phylogenetic inference problem that involves very high levels of genealogical discordance. 

The example below is a famous case where large effective population sizes (or short edge lengths) on several internal edges cause high levels of genealogical discordance such that the incorrect topology occurs more frequently than the correct topology. This scenario is termed the "anomaly zone". Phylogenetic inference methods that are consistent with the multispecies coalescent model (MSC) can correctly infer the true species tree in this scenario, whereas other methods will infer an incorrect tree.

In [7]:
# get an imbalanced species tree with crown age of 5M generations
itree = toytree.rtree.imbtree(8, treeheight=5e6)
itree.draw(ts='p', edge_type='p');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tc331cd8e64b14d3db6cb875a340d20a2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0 r1 r2 r3 r4 r5 r6 r7 0 2500000 5000000

### Set demographic parameters on the species tree 
Demographic parameters can be set on the nodes of a tree and visualized using the functions below. Here the edge lengths of the species tree are in units of **generation times** and the edge widths represent **effective population size ($N_e$)**. Using the `.set_node_values` function we set very high $N_e$ values on several internal edges.

In [8]:
# set Ne values on nodes of the tree
tree = itree.set_node_values(
    feature="Ne", 
    values={i: 2e7 for i in (9,10,11)},
    default=1e6,
)

# draw the tree showing parameters
tree.draw(ts='p', edge_type='p');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tf1c05be48d234111aeb9e8b3769d8576" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0 r1 r2 r3 r4 r5 r6 r7 0 2500000 5000000

### Genealogical discordance 
The amount of discordance on each edge of the species tree can be predicted from these parameters by calculating the *length of each edge in coalescent units*. This is measured as $t_c$ = $t_g$ / 2*$N_e$, where $t_g$ is the length in units of generation, and $t_c$ is the length in coalescent units. Below are coalescent units for the fat and thin edges above. This is shown only for demonstration purposes. In the simulation steps next we wish our tree to have edge lengths in units of generations. 

In [21]:
# short edge lengths in coalescent units
node = tree.idx_dict[10]
print("coalescent units: {:.3f}".format(node.dist / (2 * node.Ne)))

# long edge lengths in coal units
node = tree.idx_dict[8]
print("coalescent units: {:.3f}".format(node.dist / (2 * node.Ne)))

coalescent units: 0.018
coalescent units: 0.357


### Setup the simulation
Here we setup an ipcoal simulator by passing it the tree object which has edge lenths and Ne values mapped to each node. In addition, I show the mutation and recombination rates, though we could have just left them at default values. 

In [19]:
# setup a coalescent simulator
mod = ipcoal.Model(tree, mut=5e-8, recomb=1e-9)

By calling the `sim_trees()` function we can sample genealogies that evolve within the species tree container. Execute the cell below multiple times to visualize stochastic coalescent variation over multiple replicates. You should observe many deep coalescent events which cause the genealogies to frequently not match the species tree. We do this here only for visual confirmation that our model makes sense. Further below we will simulate sequence data on these genealogies.

In [26]:
# examine a single coalescent history (execute this cell multiple times)
mod.sim_trees(1)
toytree.container(mod, spacer=1, idx=0);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="tebf47bea5a864fc8b4b071e8593712d9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx=0
name=r0
Ne=1000000
t_g=714285
t_c=0.357 idx=1
name=r1
Ne=1000000
t_g=714285
t_c=0.357 idx=8
name=8
Ne=1000000
t_g=714285
t_c=0.357 idx=2
name=r2
Ne=1000000
t_g=1428571
t_c=0.714 idx=9
name=9
Ne=20000000
t_g=714285
t_c=0.018 idx=3
name=r3
Ne=1000000
t_g=2142857
t_c=1.071 idx=10
name=10
Ne=20000000
t_g=714285
t_c=0.018 idx=4
name=r4
Ne=1000000
t_g=2857142
t_c=1.429 idx=11
name=11
Ne=20000000
t_g=714285
t_c=0.018 idx=5
name=r5
Ne=1000000
t_g=3571428
t_c=1.786 idx=12
name=12
Ne=1000000
t_g=714285
t_c=0.357 idx=6
name=r6
Ne=1000000
t_g=4285714
t_c=2.143 idx=13
name=13
Ne=1000000
t_g=714285
t_c=0.357 idx=7
name=r7
Ne=1000000
t_g=5000000
t_c=2.500 idx=14
name=14 (root)
Ne=1000000
t_g=714285
t_c=0.357 r0 r1 r2 r3 r4 r5 r6 r7 0 1500000 3000000 4500000 6000000

### Infer species tree from unlinked SNPs using tetrad (SVDquartets algorithm)

In method 3 (SNPS+SVD) the tree inference problem is first decomposed into many separate quartet inference problems. Each quartet tree is inferred using a genome-wide sample of unlinked SNPs, and the estimated quartet trees are then joined together into a supertree that represents a consistent estimate of the species tree under the multispecies coalescent (MSC). This method was first developed and implemented in the **SVDquartets** software, but we will implement the same algorithm using the **tetrad** program below. 

#### Simulating SNPS
Here we will call `.sim_snps()` to sample 10K unlinked SNPs and write the result to an HDF5 formatted file. This file is then used as input to the *tetrad* program which is called from the `ipyrad.analysis (ipa)` toolkit. Finally, the inferred tree result is drawn. When the result shows up compare it will the true tree to see if it was correctly inferred. 

In [46]:
# simulate 10000 unlinked SNPs and write to a file
mod.sim_snps(10000)
mod.write_snps_to_hdf5(name='test', outdir='/tmp')

wrote 10000 SNPs to /tmp/test.snps.hdf5


#### Inferring a species tree with tetrad
As you will see, the SVDQuartets/tetrad approach is super efficient at analyzing very large genomic datasets. The shortcoming of this method, however, include that it does not scale well to super large trees (e.g., >200 species), it does not provide estimated branch lengths with the result; and it requires fairly large numbers of SNPs (e.g., thousands) to perform accurately, although this is usually not hard to obtain from empirical data. The cell below should take at most a few minutes to run. When finished, check the estimated tree that is drawn to see whether it matches the true species tree. 

In [47]:
# setup tetrad analysis
tet = ipa.tetrad(name='test', data='/tmp/test.snps.hdf5', workdir='/tmp', nboots=10)

# run distributed inference
tet.ipcluster['cores'] = 2
tet.run(auto=True, quiet=True, force=True)

# draw inferred tree
toytree.tree(tet.trees.tree).root("r7").draw(ts='s', node_labels="support");

loading snps array [8 taxa x 10000 snps]
max unlinked SNPs per quartet [nloci]: 10000
quartet sampler [full]: 70 / 70
[####################] 100% 0:00:00 | boot rep. 10 | avg SNPs/qrt: 7647 

<svg class="toyplot-canvas-Canvas" height="275.0px" id="ta9ba828bc0dd4989bb1914386f93a4b3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r1 r0 r2 r3 r4 r5 r6 r7

### Infer a tree with raxml (concatenation)
Here we simulate 1000 loci that are each 1000bp in length. This is typical of a modern phylogenomic dataset. Because each individual locus contains few variant sites, a simple approach is to combine all of the loci into a single large locus (supermatrix). 

#### Simulating concatenated loci

In [34]:
# simulate a 1000 loci each 500bp in length and write supermatrix to file
mod.sim_loci(nloci=1000, nsites=500)
mod.write_concat_to_phylip(name="test", outdir="/tmp")

wrote concat locus (8 x 5000bp) to /tmp/test.phy


#### Inferring a concatenation tree with raxml

In [40]:
# setup raxml inference command
rax = ipa.raxml(name='test', data="/tmp/test.phy", workdir="/tmp", T=1, N=10)

# run inference 
rax.run(force=True)

# draw inferred tree
toytree.tree(rax.trees.bipartitions).root("r7").draw(ts='s', node_labels="support");

job test finished successfully


<svg class="toyplot-canvas-Canvas" height="275.0px" id="t268dea3ae503440eae2c3739a43a5fc5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> nan nan nan nan nan nan nan nan 100.0 70.0 60.0 90.0 100.0 100.0 nan r5 r4 r3 r2 r1 r0 r6 r7

### Infer a species tree with ASTRAL3 (multi-step)
This involves first estimating a gene tree for every locus, and then using the gene trees as input to the astral. 

Normally, this method is quite fast and efficient, but since we are working on a small cloud-based instance for this workshop, we have few computing cores available. In testing I found the gene tree inference step below to take about 30 minutes.

In [48]:
# simulate a 1000 loci each 1000bp in length and write supermatrix to file
mod.sim_loci(nloci=1000, nsites=500)
mod.write_loci_to_hdf5(name="test", outdir="/tmp")

wrote 1000 loci to /tmp/test.seqs.hdf5


In [ ]:
# setup raxml gene tree inference for every locus.
ts = ipa.treeslider(
    name='test', 
    data="/tmp/test.seqs.hdf5", 
    workdir="/tmp",
    inference_args={'f': 'd', 'x': None, "N": 10},
)
ts.run(auto=True, force=True)

building database: nwindows=1000; minsnps=1
[###############     ]  77% 0:08:25 | inferring trees 

In [37]:
# setup astral inference from inferred gene trees
ast = ipa.astral(name='test', data="...", workdir="/tmp")
ast.run()

# draw the inferred species tree
toytree.tree(ast.tree).draw();

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t87b563a6a8004c87ab8edf165ca9bcf2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r7 r6 r5 r4 r3 r2 r0 r1